## Imports

In [44]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd

from pandas_geojson import to_geojson, write_geojson
from geopy.geocoders import GoogleV3
from textwrap import shorten

## Read-in

In [45]:
df = pd.read_csv('Redevelopment of South Florida Religious Real Estate  - Sheet1.csv')
df = df[df['Unnamed: 17'] != 'DO NOT INCLUDE']
df = df.rename(columns={'Property name/Church on site':'Property_name_Church_on_site',
                       'Developer (s)':'Developers',
                       'Project (Number of buildings, height, units, sf)':'Project_details'})

In [46]:
df = df.fillna('No_text')

## Google Maps API Geolocater Setup

In [47]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [48]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['Address'].apply(geocode)

In [49]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Write to GeoJSON

In [50]:
df.columns

Index(['Property_name_Church_on_site', 'Developers', 'Project name ',
       'Address', 'Project_details', 'Project status',
       'Plan for church, synagogue', 'More info ', 'Link', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Who reached out? ', 'Unnamed: 17',
       'geocoded', 'lat', 'lon'],
      dtype='object')

In [52]:
geo_json = to_geojson(df=df, lat='lat', lon='lon',
                     properties=['Property_name_Church_on_site','Developers','Project name ',
                                 'Address','Project_details',
                                'Project status','Plan for church, synagogue'])
write_geojson(geo_json, filename='SoFlaChurches.geojson', indent=4)

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [55]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/SoFla_Churches_Redevelopment'
print(base_name)

https://trd-digital.github.io/trd-news-interactive-maps/SoFla_Churches_Redevelopment
